<h1>Citibike Network Assignment</h1>
<li>The file, 201809-citibike-tripdata.csv, contains citibike trip data from September 2018
<li>The data:<br>
"tripduration","starttime","stoptime","start station id","start station name","start station latitude","start station longitude","end station id","end station name","end station latitude","end station longitude","bikeid","usertype","birth year","gender"
<li>Each record in the data is a trip 
<li>The data is described at https://www.citibikenyc.com/system-data

<h1>STEP 1: Read the data into a dataframe</h1>
<li>Convert station ids to str if necessary

In [1]:
import pandas as pd
import numpy as np
datafile = "/Users/hardeepjohar/Documents/Courses/DataAnalyticsFall2017/networks/2014-01 - Citi Bike trip data.csv"
df = pd.read_csv(datafile)


<h1>STEP 2: Remove data that have any nans in any row</h1>
<li>and convert stationids to str 

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300400 entries, 0 to 300399
Data columns (total 15 columns):
tripduration               300400 non-null int64
starttime                  300400 non-null object
stoptime                   300400 non-null object
start station id           300400 non-null int64
start station name         300400 non-null object
start station latitude     300400 non-null float64
start station longitude    300400 non-null float64
end station id             300400 non-null int64
end station name           300400 non-null object
end station latitude       300400 non-null float64
end station longitude      300400 non-null float64
bikeid                     300400 non-null int64
usertype                   300400 non-null object
birth year                 300400 non-null object
gender                     300400 non-null int64
dtypes: float64(4), int64(5), object(6)
memory usage: 34.4+ MB


In [ ]:
df[(df['start station id'] == '83') & (df['end station id']=='540')]

In [3]:
df.dropna(inplace=True)
df['start station id'] = df['start station id'].apply(str)
df['end station id'] = df['end station id'].apply(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300400 entries, 0 to 300399
Data columns (total 15 columns):
tripduration               300400 non-null int64
starttime                  300400 non-null object
stoptime                   300400 non-null object
start station id           300400 non-null object
start station name         300400 non-null object
start station latitude     300400 non-null float64
start station longitude    300400 non-null float64
end station id             300400 non-null object
end station name           300400 non-null object
end station latitude       300400 non-null float64
end station longitude      300400 non-null float64
bikeid                     300400 non-null int64
usertype                   300400 non-null object
birth year                 300400 non-null object
gender                     300400 non-null int64
dtypes: float64(4), int64(3), object(8)
memory usage: 36.7+ MB


<h1>STEP 3: Write a function that returns a graph given a citibike data frame</h1> 
<li>Your function should return two things:
<ol>
<li>a graph
<li>a dictionary with station ids as the key and station name as the value
</ol>
<li>The graph should contain 
<ol>
<li>nodes (station ids)
<li>edges (station id, station id)
<li>edge data 
<ol>
<li>count: number of trips on the edge
<li>time: average duration - pickup to dropoff - on that edge
</ol>
</ol>
<li><b>Note:</b> the edge (x1,y1) is the same as (y1,x1) even though the start station ids and end station ids are flipped in the dataframe

In [15]:
G= nx.Graph()
row = df.iloc[0]
start = row['start station id']
start

'2009'

In [5]:
def get_citibike_graph(df):
    import networkx as nx
    G = nx.Graph()
    node_names = dict()
    edges = dict()
    for ind in df.index:
        row = df.loc[ind]
        start = row['start station id']
        end = row['end station id']
        node_names[start] = row['start station name']
        node_names[end] = row['end station name']
        #Arrange start and end in ascending order (or edges dict won't match)
        start,end = max(start,end),min(start,end)
        time = row['tripduration']
        if edges.get((start,end)):
            cnt = edges[(start,end)][0]+1
            time = time + edges[(start,end)][1]           
            edges[(start,end)] = (cnt,time)
        else:
            edges[(start,end)] = (1,time)
        G.add_edge(start,end,trips=edges[(start,end)][0],time=edges[(start,end)][1]/edges[(start,end)][0])
    #YOUR CODE GOES HERE
    return G,node_names
    

<h1>STEP 4: Create the following graphs using the function above</h1>
<li>G: A graph of all the data in the dataframe
<li>m_G: A graph containing only data from male riders
<li>f_G: A graph containing only data from female riders
<li>Note: for m_G and f_G you will need to extract data from the dataframe

In [6]:
G,nodes=get_citibike_graph(df)

In [7]:
maledf = df[df['gender']==1]
femaledf = df[df['gender']==2]
f_G = get_citibike_graph(femaledf)[0]
m_G = get_citibike_graph(maledf)[0]